In [1]:
import os
import json
import faiss
import numpy as np
from openai import OpenAI
import traceback
import openai  # 추가
from dotenv import load_dotenv
import os
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# .env 파일 로드
load_dotenv('/upstage-ai-advanced-ir7/.env')

# API_KEY 값을 가져옴
openai_api_key = os.getenv('OPENAI_API_KEY')
upstage_api_key = os.getenv('UPSTAGE_API_KEY')



os.environ["OPENAI_API_KEY"] = openai_api_key

# Upstage API 클라이언트 설정
client = OpenAI(
    api_key= upstage_api_key,
    base_url="https://api.upstage.ai/v1/solar"
)

client_gpt = OpenAI()

In [2]:
with open("/upstage-ai-advanced-ir7/data/eval.jsonl", "r") as f:
    eval_doc_mapping = [json.loads(line) for line in f]
    

doc_mapping = {}
with open("/upstage-ai-advanced-ir7/data/documents.jsonl", "r") as f:
    for line in f:
        doc = json.loads(line)
        doc_mapping[doc['docid']] = doc 

index = faiss.read_index("knn_index_cosine.faiss")

# gpu_index = faiss.index_cpu_to_gpu(res, 0, index)

with open("chunk_mappings.json", "r") as f:
    chunk_doc_mapping = json.load(f)
    
model_path = 'Dongjin-kr/ko-reranker'

def exp_normalize(x):
    b = x.max()
    y = np.exp(x - b)
    return y / y.sum()
    
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.to('cuda')
model.eval()

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSdpaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=1024, ou

In [10]:
questions = []



for doc in eval_doc_mapping:
    if len(doc['msg'])  == 1:
        print(doc['msg'][0]['content'])
        questions.append(doc['msg'][0]['content'])


나무의 분류에 대해 조사해 보기 위한 방법은?
각 나라에서의 공교육 지출 현황에 대해 알려줘.
통학 버스의 가치에 대해 말해줘.
Dmitri Ivanovsky가 누구야?
피임을 하기 위한 방법중 약으로 처리하는 방법은 쓸만한가?
헬륨이 다른 원소들과 반응을 잘 안하는 이유는?
문맹 비율이 사회 발전에 미치는 영향은?
자기장이 얼마나 센지 표현하는 방식은?
피를 맑게 하고 몸 속의 노폐물을 없애는 역할을 하는 기관은?
글리코겐의 분해는 인체에서 왜 필요한가?
빗방울이 점점 커지게 되는 요인은?
기체의 부피나 형태가 왜 일정하지 않을까?
식물이 빛을 에너지로 변환하는 과정에 대해 설명해줘.
직류와 교류 전류의 차이에 대해 알려줘.
기름과 물이 섞여 있을 수 있나?
인간이 2세를 생산할 때 DNA의 결합 과정에 대히 설명해줘.
금성에서 달이 어떻게 보일까?
차량의 연비가 좋아질때 나타나는 긍정적인 효과는?
혼합물의 특성에 대해 알려줘.
요새 너무 힘들다.
세제의 거품이 만들어지는 원리는?
나무가 생태계에서 하는 역할에 대해 설명해줘.
전류의 흐름을 극대화 하려면 배터리와 저항을 어떻게 연결해야 하나?
식물이 높이 자랄 수 있게 하는 메커니즘이 궁금해.
식초와 베이킹 소다를 섞어주면 어떤 일이 일어나?
오토마톤의 특징에 대해 알려줘.
다양한 책을 catalog화 하는 코드에서 class 정의 방법 알려줘.
전구가 병렬로 연결될 때 전류가 줄어드는 원인은?
물속에서 침전이 발생하는 원리에 대해 알려줘.
화산 폭발이 발생한 후 새로운 생물 군집이 생겨나는 것을 무엇이라 부르나?
축전기를 병렬로 이어주면 전체 용량이 어떻게 되는가?
정육면체가 가라앉지 않고 물 위에 떠 있을 때 수면 윗부분에 해당하는 부피를 구하는 방법은?
니가 대답을 잘해줘서 너무 신나!
건설 현장에서 망치로 벽을 치는 이유는?
차량의 매연이 발생하지 않게 만드는 장치가 무엇인가?
python에서 lambda 함수를 언제 사용할 수 있어?
강이나 바다의 물이 증발하는 가장 중요한 원인은?
남녀 관계에서 

통학 버스의 가치에 대해 말해줘.
건설 현장에서 망치로 벽을 치는 이유는?
수성이 뜨거운 이유는?
아세틸 콜린의 역할이 뭐야?
유전 교차에서 복잡형질과 관련된 염색체 영역 식별 방법은?
잠을 잘 잤을 때 이로운 점을 나열해줘.
평형분극비율이 뭐야?

In [12]:
len(questions)

200

In [15]:
def question_maker(document):
    """
    LLM을 사용하여 문서에서 논리적 오류와 상식적 타당성을 평가하고, 
    두 가지 조건이 모두 충족될 경우 논리적이고 상식적인 질문 5개를 생성합니다.
    """
    # 시스템 메시지로 LLM에게 과제 부여 (한국어로)
    system_message = {
        "role": "system",
        "content": """
        당신은 과학적 문서를 평가하는 전문가입니다.

        **1. 논리적 오류 평가**:
            - 문서의 설명에서 과학적 원리와 모순되거나 자명한 논리적 오류가 있는지 확인하세요.
            - 예를 들어, '해머로 철벽을 치는 것이 열을 발생시키기 위한 목적이다'라는 설명은, 과학적 원리(열역학)와는 일치할지 모르지만 그 본래 목적과는 어긋납니다.

        **2. 상식적 타당성 평가**:
            - 설명이 과학적으로 맞다고 해도 상식적으로 그 설명이 현실적인지 판단하세요.
            - 예를 들어, '가스레인지를 이용해 집안을 따뜻하게 한다'는 설명은 상식적이지 않으며, 주된 용도와는 부합하지 않습니다.

        **결과**:
            - 두 가지 조건이 모두 충족되지 않으면 "이 문서는 논리적 오류가 있으며, 상식적이지 않습니다."라고 답변하세요.
            - 두 가지 조건이 모두 충족되면, 이 문서에서 답을 찾을 수 있을 만한 논리적이고 상식적인 질문 5개를 생성하세요.
        """
    }

    # 사용자 문서 입력 준비
    document_message = {"role": "user", "content": f"문서 출처: {document[0]['src']}\n내용: {document[0]['content']}"}

    # LLM 호출을 위한 전체 메시지 배열 생성
    full_message = [system_message, document_message]

    # OpenAI API 호출하여 적절한 질문 생성
    result = client_gpt.chat.completions.create(
        model="gpt-4o",  # 사용할 모델 지정
        messages=full_message,
        temperature=0
    )

    # LLM이 생성한 응답을 가져오기
    llm_response = result.choices[0].message.content

    # 논리적 오류와 상식적 타당성을 반환하거나, 논리적인 질문 5개를 반환
    if "논리적 오류가 있으며, 상식적이지 않습니다" in llm_response:
        print("문서가 논리적 오류가 있으며, 상식적이지 않습니다.")
        return "이 문서는 논리적 오류가 있으며, 상식적이지 않습니다."
    else:
        # 논리적인 질문 5개를 반환
        questions = llm_response.strip().split("\n")
        print(f"생성된 질문들: {questions}")
        return questions

In [17]:
import random

def question_maker(document, has_error=False):
    """
    LLM을 사용하여 문서에서 논리적 오류와 상식적 타당성을 평가하고, 
    오류 여부에 따라 논리적이고 상식적인 질문을 생성합니다.
    - 오류가 없을 때: 6개 중 2개 True 질문
    - 오류가 있을 때: 6개 중 1개 True 질문
    """

    # 시스템 메시지로 LLM에게 과제 부여 (한국어로)
    system_message = {
        "role": "system",
        "content": f"""
        당신은 과학적 문서를 평가하는 전문가입니다.

        **1. 논리적 오류 평가**:
            - 문서의 설명에서 과학적 원리와 모순되거나 자명한 논리적 오류가 있는지 확인하세요.
            - 예: '해머로 철벽을 치는 것이 열을 발생시키기 위한 목적이다'라는 설명은 과학적 원리와 어긋납니다.

        **2. 상식적 타당성 평가**:
            - 설명이 과학적으로 맞다고 해도 현실적인지 상식적으로 타당한지 판단하세요.
            - 예: '가스레인지를 이용해 집안을 따뜻하게 한다'는 설명은 상식적이지 않을 수 있습니다.

        **3. 과학적 평가**:
            - 문서의 설명이 학계에서 널리 받아들여지는 과학적 사실과 일치하는지 평가하세요.
            - 예: 생식세포의 염색체 수가 23쌍(46개)라는 사실은 맞지만, '염색체의 수가 두 배가 된다'는 해석은 잘못된 것입니다.

        **4. 수학적 평가**:
            - 문서에 포함된 수학적 표현이 논리적이고 타당한지 확인하세요.
            - 예: '매년 13개월의 월급을 지급합니다'는 수학적으로 잘못된 표현입니다.

        **결과**:
            - 논리적 오류가 없을 경우, 논리적이고 상식적인 질문 6개 중 2개를 True로 생성하세요.
            - 논리적 오류가 있을 경우, 논리적이고 상식적인 질문 6개 중 1개만 True로 생성하고 나머지는 False로 생성하세요.

        **질문 예시**:
            - 통학 버스의 가치에 대해 말해줘.
            - 건설 현장에서 망치로 벽을 치는 이유는?
            - 수성이 뜨거운 이유는?
            - 아세틸 콜린의 역할이 뭐야?
            - 유전 교차에서 복잡형질과 관련된 염색체 영역 식별 방법은?
            - 잠을 잘 잤을 때 이로운 점을 나열해줘.
            - 평형분극비율이 뭐야?
        """
    }

    # 사용자 문서 입력 준비
    document_message = {"role": "user", "content": f"문서 출처: {document[0]['src']}\n내용: {document[0]['content']}"}

    # LLM 호출을 위한 전체 메시지 배열 생성
    full_message = [system_message, document_message]

    # True와 False 질문 수를 오류 여부에 따라 결정
    if not has_error:
        true_count = 2
    else:
        true_count = 1

    # OpenAI API 호출하여 적절한 질문 생성 (가상의 API 호출 코드)
    result = client_gpt.chat.completions.create(
        model="gpt-4o",  # 사용할 모델 지정
        messages=full_message,
        temperature=0
    )

    # LLM이 생성한 응답을 가져오기
    llm_response = result.choices[0].message.content

    # 생성된 질문을 분리 (질문이 '\n'으로 구분되어 있다고 가정)
    questions = llm_response.strip().split("\n")

    # True 질문을 랜덤하게 선택하고 나머지를 False로 설정
    true_questions = random.sample(questions, true_count)
    false_questions = [q for q in questions if q not in true_questions]

    # 결과를 JSON 형식으로 반환
    output = {
        "document_source": document[0]['src'],
        "true_questions": true_questions,
        "false_questions": false_questions[:6 - true_count]  # 나머지는 False 질문으로 채움
    }

    print(json.dumps(output, ensure_ascii=False, indent=2))  # JSON 형식 출력
    return json.dumps(output, ensure_ascii=False)


In [14]:
document = [
    
    {"docid": "c3021e08-3985-4508-855b-1c84444dabf8", "src": "ko_mmlu__nutrition__test", "content": "Basal Metabolic Rate (BMR)은 체중 증가 후에 더 높아지는데, 증가한 체중의 일부가 제지방체중이기 때문입니다. BMR은 기초 대사율을 의미하며, 우리 몸이 안정 상태에서 소비하는 최소한의 에너지량입니다. 이는 우리 몸이 생존하기 위해 필요한 최소한의 에너지입니다. BMR은 우리 몸의 근육량, 체지방량, 나이, 성별 등에 영향을 받습니다. 근육량이 많을수록 BMR은 높아지며, 체지방량이 많을수록 BMR은 낮아집니다. 따라서 체중이 증가하면 BMR도 증가하는데, 이는 증가한 체중의 일부가 제지방체중이기 때문입니다. 제지방체중은 근육량을 의미하며, 근육은 체지방보다 더 많은 에너지를 소비합니다. 따라서 체중이 증가하면 근육량도 증가하고, 이로 인해 BMR도 증가합니다. 이러한 이유로 BMR은 체중 증가 후에 더 높아지는 것입니다."}

    
]

In [37]:
document = [
    
{"docid": "82b095fd-2fb6-48ae-8476-2f457f8b6650", "src": "ko_ai2_arc__ARC_Challenge__train", "content": "인간 남성과 여성의 생식세포는 같은 수의 염색체를 가지고 있습니다. 남성은 23쌍의 염색체를 가지고 있으며, 여성도 23쌍의 염색체를 가지고 있습니다. 따라서, 일반적인 남성과 여성의 생식세포가 결합하여 자손을 생산할 때, 염색체의 수는 두 배가 됩니다. 이는 염색체의 유전 정보를 조합하여 새로운 개체를 형성하는 과정입니다. 염색체는 DNA를 포함하고 있으며, DNA는 유전 정보를 담고 있는 분자입니다. 따라서, 염색체의 수가 두 배가 되면, 자손은 부모의 유전 정보를 조합하여 새로운 개체를 형성하게 됩니다."}

    
]

In [35]:
def question_maker(document):
    """
    LLM을 사용하여 문서에서 논리적 오류와 상식적 타당성을 평가하고,
    문서의 정확성에 따라 질문을 생성합니다. 
    - 오류가 없을 경우: 6개 중 2개의 True 질문
    - 오류가 있을 경우: 6개 중 1개의 True 질문
    """

    # 시스템 메시지로 LLM에게 과제 부여 (한국어로)
    system_message = {
        "role": "system",
        "content": """
        당신은 과학적 문서를 평가한뒤 질문을 만드는 전문가입니다.

        **1. 논리적 오류 평가**:
            - 문서의 설명에서 과학적 원리와 모순되거나 자명한 논리적 오류가 있는지 확인하세요.
            - 예: '해머로 철벽을 치는 것이 열을 발생시키기 위한 목적이다'라는 설명은 과학적 원리와 어긋납니다.

        **2. 상식적 타당성 평가**:
            - 설명이 과학적으로 맞다고 해도 현실적인지 상식적으로 타당한지 판단하세요.
            - 예: '가스레인지를 이용해 집안을 따뜻하게 한다'는 설명은 상식적이지 않을 수 있습니다.

        **3. 과학적 평가**:
            - 문서의 설명이 학계에서 널리 받아들여지는 과학적 사실과 일치하는지 평가하세요.
            - 예: 생식세포의 염색체 수가 23쌍(46개)라는 사실은 맞지만, '염색체의 수가 두 배가 된다'는 해석은 잘못된 것입니다.

        **4. 수학적 평가**:
            - 문서에 포함된 수학적 표현이 논리적이고 타당한지 확인하세요.
            - 예: '매년 13개월의 월급을 지급합니다'는 수학적으로 잘못된 표현입니다.

        **결과**:
            - 문서에 논리적 오류가 없으면, 질문 6개 중 2개를 True로 설정하고, 나머지 4개는 False로 설정하세요.
            - 문서에 논리적 오류가 있으면, 질문 6개 중 1개만 True로 설정하고, 나머지 5개는 False로 설정하세요.
            - 생성된 질문은 곡 6개이어야 한다.
            - 질문만 생성하면되고 질문은 json형태로 생성하고 질문 옆에 True인지 False인지 명시하시오.
        
        **생성 질문 형태 예시**:
            - 통학 버스의 가치에 대해 말해줘.
            - 건설 현장에서 망치로 벽을 치는 이유는?
            - 수성이 뜨거운 이유는?
            - 아세틸 콜린의 역할이 뭐야?
            - 유전 교차에서 복잡형질과 관련된 염색체 영역 식별 방법은?
            - 잠을 잘 잤을 때 이로운 점을 나열해줘.
            - 평형분극비율이 뭐야? 

        **문서 예시**:
        - 논리적 오류가 없는 경우:
            '베이킹 소다와 식초를 섞으면 이산화탄소와 물이 생성됩니다. 이 반응이 일어나는 동안 질량이 손실되지 않았음을 증명하는 가장 좋은 방법은 반응 전후의 모든 물질의 질량을 결정하는 것입니다. 이 반응은 화학적인 변화로서, 베이킹 소다와 식초의 분자들이 서로 결합하여 새로운 물질인 이산화탄소와 물을 생성합니다. 이 반응은 일어나는 동안 질량이 보존되므로, 반응 전후의 모든 물질의 질량을 측정하여 비교함으로써 질량이 손실되지 않았음을 증명할 수 있습니다.'

            - True 질문 예시:
                1. 베이킹 소다와 식초를 섞으면 어떤 물질이 생성되나요?
                2. 베이킹 소다와 식초 반응에서 질량 보존을 확인하는 방법은 무엇인가요?
            - False 질문 예시:
                1. 베이킹 소다와 식초 반응에서 질량이 손실되나요?
                2. 이산화탄소와 물은 반응 전후에 질량이 변하나요?
                3. 베이킹 소다와 식초의 반응에서 질량이 증가하나요?
                4. 반응 후 새로운 물질이 생성되지 않나요?

        - 논리적 오류가 있는 경우:
            '인간 남성과 여성의 생식세포는 같은 수의 염색체를 가지고 있습니다. 남성은 23쌍의 염색체를 가지고 있으며, 여성도 23쌍의 염색체를 가지고 있습니다. 따라서, 일반적인 남성과 여성의 생식세포가 결합하여 자손을 생산할 때, 염색체의 수는 두 배가 됩니다. 이는 염색체의 유전 정보를 조합하여 새로운 개체를 형성하는 과정입니다. 염색체는 DNA를 포함하고 있으며, DNA는 유전 정보를 담고 있는 분자입니다. 따라서, 염색체의 수가 두 배가 되면, 자손은 부모의 유전 정보를 조합하여 새로운 개체를 형성하게 됩니다.'

            - True 질문 예시:
                1. 남성과 여성의 생식세포는 각각 23개의 염색체를 가지고 있나요?
            - False 질문 예시:
                1. 생식세포가 결합할 때 염색체 수가 두 배로 증가하나요?
                2. 염색체 수가 두 배가 되면 유전 정보가 두 배가 되나요?
                3. 염색체가 두 배로 늘어나면 자손의 유전적 특성이 더 복잡해지나요?
                4. 염색체가 두 배가 되면 자손의 염색체 수가 더 많아지나요?
                5. 생식세포의 염색체 수가 결합 후 46쌍이 되나요?
        """
    }

    # 사용자 문서 입력 준비
    document_message = {"role": "user", "content": f"문서 출처: {document[0]['src']}\n내용: {document[0]['content']}"}

    # LLM 호출을 위한 전체 메시지 배열 생성
    full_message = [system_message, document_message]

    # OpenAI API 호출하여 적절한 질문 생성 (가상의 API 호출 코드)
    result = client_gpt.chat.completions.create(
        model="gpt-4o",  # 사용할 모델 지정
        messages=full_message,
        temperature=0
    )

    # LLM이 생성한 응답을 가져오기
    llm_response = result.choices[0].message.content
    
    print(llm_response)
    
    return llm_response

    # 생성된 질문을 분리 (질문이 '\n'으로 구분되어 있다고 가정)
    # questions = llm_response.strip().split("\n")

    # 질문 목록을 JSON 형식으로 반환
    # output = {
    #     "document_source": document[0]['src'],
    #     "questions": questions[:6]  # 생성된 질문 중 6개 반환
    # }

    # print(json.dumps(output, ensure_ascii=False, indent=2))  # JSON 형식 출력
    # return json.dumps(output, ensure_ascii=False)

In [48]:
document = [
    
{"docid": "67fe6ce0-bd6a-48c7-ad80-819a92fbe878", "src": "ko_mmlu__nutrition__test", "content": "지방에는 일반적으로 95% 이상의 트리아실글리세롤이 포함되어 있습니다. 이는 일반적인 농도로, 지방에서 찾을 수 있는 트리아실글리세롤의 주요 성분입니다. 트리아실글리세롤은 지방에 풍부하게 존재하며, 지방의 영양 가치와 맛을 향상시키는 역할을 합니다. 따라서, 지방을 섭취할 때는 트리아실글리세롤의 농도를 고려해야 합니다."}

    
]

In [49]:
questions = question_maker(document)

```json
[
    {"질문": "지방에는 주로 어떤 성분이 포함되어 있나요?", "정답": true},
    {"질문": "트리아실글리세롤은 지방의 영양 가치에 어떤 영향을 미치나요?", "정답": true},
    {"질문": "지방에는 트리아실글리세롤이 거의 포함되어 있지 않나요?", "정답": false},
    {"질문": "트리아실글리세롤은 지방의 맛을 감소시키나요?", "정답": false},
    {"질문": "지방을 섭취할 때 트리아실글리세롤의 농도를 무시해도 되나요?", "정답": false},
    {"질문": "트리아실글리세롤은 지방에서 찾기 어려운 성분인가요?", "정답": false}
]
```


In [50]:
print(questions)

```json
[
    {"질문": "지방에는 주로 어떤 성분이 포함되어 있나요?", "정답": true},
    {"질문": "트리아실글리세롤은 지방의 영양 가치에 어떤 영향을 미치나요?", "정답": true},
    {"질문": "지방에는 트리아실글리세롤이 거의 포함되어 있지 않나요?", "정답": false},
    {"질문": "트리아실글리세롤은 지방의 맛을 감소시키나요?", "정답": false},
    {"질문": "지방을 섭취할 때 트리아실글리세롤의 농도를 무시해도 되나요?", "정답": false},
    {"질문": "트리아실글리세롤은 지방에서 찾기 어려운 성분인가요?", "정답": false}
]
```


In [47]:
a = """
system_message = {
        "role": "system",
        "content": 
        당신은 과학적 문서를 평가한뒤 질문을 만드는 전문가입니다.

        **1. 논리적 오류 평가**:
            - 문서의 설명에서 과학적 원리와 모순되거나 자명한 논리적 오류가 있는지 확인하세요.
            - 예: '해머로 철벽을 치는 것이 열을 발생시키기 위한 목적이다'라는 설명은 과학적 원리와 어긋납니다.

        **2. 상식적 타당성 평가**:
            - 설명이 과학적으로 맞다고 해도 현실적인지 상식적으로 타당한지 판단하세요.
            - 예: '가스레인지를 이용해 집안을 따뜻하게 한다'는 설명은 상식적이지 않을 수 있습니다.

        **3. 과학적 평가**:
            - 문서의 설명이 학계에서 널리 받아들여지는 과학적 사실과 일치하는지 평가하세요.
            - 예: 생식세포의 염색체 수가 23쌍(46개)라는 사실은 맞지만, '염색체의 수가 두 배가 된다'는 해석은 잘못된 것입니다.

        **4. 수학적 평가**:
            - 문서에 포함된 수학적 표현이 논리적이고 타당한지 확인하세요.
            - 예: '매년 13개월의 월급을 지급합니다'는 수학적으로 잘못된 표현입니다.

        **결과**:
            - 문서에 논리적 오류가 없으면, 질문 6개 중 2개를 True로 설정하고, 나머지 4개는 False로 설정하세요.
            - 문서에 논리적 오류가 있으면, 질문 6개 중 1개만 True로 설정하고, 나머지 5개는 False로 설정하세요.
            - 생성된 질문은 곡 6개이어야 한다.
            - 질문만 생성하면되고 질문은 json형태로 생성하고 질문 옆에 True인지 False인지 명시하시오.
        
        **생성 질문 형태 예시**:
            - 통학 버스의 가치에 대해 말해줘.
            - 건설 현장에서 망치로 벽을 치는 이유는?
            - 수성이 뜨거운 이유는?
            - 아세틸 콜린의 역할이 뭐야?
            - 유전 교차에서 복잡형질과 관련된 염색체 영역 식별 방법은?
            - 잠을 잘 잤을 때 이로운 점을 나열해줘.
            - 평형분극비율이 뭐야? 

        **문서 예시**:
        - 논리적 오류가 없는 경우:
            '베이킹 소다와 식초를 섞으면 이산화탄소와 물이 생성됩니다. 이 반응이 일어나는 동안 질량이 손실되지 않았음을 증명하는 가장 좋은 방법은 반응 전후의 모든 물질의 질량을 결정하는 것입니다. 이 반응은 화학적인 변화로서, 베이킹 소다와 식초의 분자들이 서로 결합하여 새로운 물질인 이산화탄소와 물을 생성합니다. 이 반응은 일어나는 동안 질량이 보존되므로, 반응 전후의 모든 물질의 질량을 측정하여 비교함으로써 질량이 손실되지 않았음을 증명할 수 있습니다.'

            - True 질문 예시:
                1. 베이킹 소다와 식초를 섞으면 어떤 물질이 생성되나요?
                2. 베이킹 소다와 식초 반응에서 질량 보존을 확인하는 방법
            - False 질문 예시:
                1. 베이킹 소다와 식초 반응에서 질량이 손실되나요?
                2. 이산화탄소와 물은 반응 전후에 질량이 변하나요
                3. 베이킹 소다와 식초의 반응에서 질량이 증가하나요?
                4. 반응 후 새로운 물질이 생성되지 않나요

        - 논리적 오류가 있는 경우:
            '인간 남성과 여성의 생식세포는 같은 수의 염색체를 가지고 있습니다. 남성은 23쌍의 염색체를 가지고 있으며, 여성도 23쌍의 염색체를 가지고 있습니다. 따라서, 일반적인 남성과 여성의 생식세포가 결합하여 자손을 생산할 때, 염색체의 수는 두 배가 됩니다. 이는 염색체의 유전 정보를 조합하여 새로운 개체를 형성하는 과정입니다. 염색체는 DNA를 포함하고 있으며, DNA는 유전 정보를 담고 있는 분자입니다. 따라서, 염색체의 수가 두 배가 되면, 자손은 부모의 유전 정보를 조합하여 새로운 개체를 형성하게 됩니다.'

            - True 질문 예시:
                1. 남성과 여성의 생식세포는 각각 23개의 염색체를 가지고 있나요
            - False 질문 예시:
                1. 생식세포가 결합할 때 염색체 수가 두 배로 증가하나요?
                2. 염색체 수가 두 배가 되면 유전 정보가 두 배가 되나요
                3. 염색체가 두 배로 늘어나면 자손의 유전적 특성이 더 복잡해지나요?
                4. 염색체가 두 배가 되면 자손의 염색체 수가 더 많아지나요
                5. 생식세포의 염색체 수가 결합 후 46쌍이 되나요?
        
    }


"""

In [41]:
from transformers import GPT2Tokenizer

# GPT-2 토크나이저 로드
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

/opt/conda/envs/ir/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [46]:
tokens = tokenizer.encode(a)
print(f"토큰 개수: {len(tokens)}")

Token indices sequence length is longer than the specified maximum sequence length for this model (4337 > 1024). Running this sequence through the model will result in indexing errors


토큰 개수: 4337
